## 函数调用

参考：[assistant_add_custom_tool.py](https://github.com/QwenLM/Qwen-Agent/blob/main/examples/assistant_add_custom_tool.py)

In [1]:
import json
import json5

from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

注册函数

In [2]:
@register_tool('get_nickname')
class NicknameTool(BaseTool):
    description = '查询用户的小名'
    parameters = [{
        'name': 'name',
        'type': 'string',
        'description': '用户的名字',
        'required': True,
    }]

    def call(self, params: str, **kwargs) -> str:
        name = json5.loads(params)['name']
        nickname = name + '宝宝'
        return json.dumps({'nickname': nickname}, ensure_ascii=False)

为 Agent 添加函数

In [3]:
# Postgres Agent 的系统指令
SYSTEM_PROMPT = "你是一个AI助手"

llm_cfg = {
    'model': 'Qwen3-0.6B-FP8',
    'model_server': 'http://localhost:8000/v1',
    'api_key': 'token-kcgyrk',
    'generate_cfg': {
        'top_p': 0.95,
        'temperature': 0.6,
    }
}

tools = ['get_nickname']
bot = Assistant(
    llm=llm_cfg,
    name='小名查询助手',
    description='查询用户的小名',
    system_message=SYSTEM_PROMPT,
    function_list=tools,
)

在与 llm 的对话中调用函数

In [4]:
query = '用户Alex的小名是什么？'
messages = [{'role': 'user', 'content': query}]

response = bot.run_nonstream(messages)
response[-1]

{'role': 'assistant', 'content': '\n\nAlex宝宝', 'name': '小名查询助手'}

In [5]:
print(response[-1].get('content').strip())

Alex宝宝
